# Hello World - Geting started with GlassFlow

This example demonstrates how to get started with GlassFlow. This creates a hello-world pipeline on GlassFlow and uses the python SDK to send data to the pipeline 

## Pre-requisites

- Create your free GlassFlow account via the [GlassFlow WebApp](https://app.glassflow.dev).
- Get your [Personal Access Token](https://app.glassflow.dev/profile) to authorize the Python SDK to interact with GlassFlow Cloud.


In [ ]:
%pip install "glassflow>=2.0.8" Faker

In [ ]:
import glassflow

In [ ]:
# fill credentials
# Please edit this variable with your own personal access token from https://app.glassflow.dev/profile
personal_access_token = "cZQvEHefrfNhcthYyarzyJnQQqQBnR5nRfBdBjG5QwV2gbMEtVYFRj7EFGnEYQprKKhKFrhSx4pKxyD25jrMCmM4fXJTf7TbbPBhmEmpH9BznsYQVhWn3F8NTnCYhd7w"

In [ ]:
client = glassflow.GlassFlowClient(
    personal_access_token=personal_access_token
)

## Create / Set up a space 
Spaces are a way to organize pipelines. In this example, we will use a space called `examples`.

In [ ]:
# Get the space named "examples" (or create one if no space is found)
list_spaces = client.list_spaces()

space_name = "examples"
for s in list_spaces.spaces:
    if s["name"] == space_name:
        space = glassflow.Space(
            personal_access_token=client.personal_access_token,
            id=s["id"], 
            name=s["name"]
        )
        break
else:
    space = client.create_space(name=space_name)

print(f"Space \"{space.name}\" with ID: {space.id}")

## Create Pipeline 
Pipeline consists of 3 major components: 
1. *Source* : Source defines how data enters the pipeline. There are several managed sources or you can use the GlassFlow python SDK to send data to the pipeline. In this example, we will use the python SDK (that is the default choice when setting up the pipeline).
2. *Transform* : This is the python code along with requirements and env variables which define  the function that should be executed for every data entering the pipeline.
3. *Sink*: Sink defines where the data should end up after transformation in GlassFlow. There are several managed sinks or you can use the GlassFlow python SDK to pull data from the pipeline. In this example, we will use a managed webhook sink.


You can also use the web-application at https://app.glassflow.dev to create the pipeline. 
In that case, skip to the section [Send Data](#Send-data-to-the-pipeline)


In [ ]:
# Pipeline parameters 
pipeline_name = "hello-glassflow"
space_id = space.id
# Source 

# Transform
requirements_text = ""
with open("requirements.txt") as f:
    requirements_txt = f.read()
transformation_file = 'transform.py' # the location of the python code

# Sink
# Define the webhook url where the transformed data should be sent by GlassFlow. You can use a test sink url from 
# https://webhook.glassflow.xyz/ or https://webhook.site/

sink_kind="webhook"
sink_config={
    "url": "https://webhook.glassflow.xyz/78f22b0f-aa69-490b-a3b3-5dc79086be61", # example webhook sink
    "method": "POST",
    }

In [ ]:
pipeline = client.create_pipeline(
    name=pipeline_name, 
    transformation_file=transformation_file,
    space_id=space_id, 
    requirements=requirements_txt,
    sink_kind=sink_kind,
    sink_config=sink_config
)
print("Pipeline ID:", pipeline.id)

## Send data to the pipeline 

In [ ]:
# pipeline id (replace with your pipeline id in case you used an existing pipeline or created one via the web-app
pipeline_id = pipeline.id 

In [ ]:
from faker import Faker

def random_datagen():
    fake = Faker()
    data = {
        "name": fake.name(),
        "age": fake.random_int(18, 100),
        "country": fake.country()
    }
    return data

In [ ]:
# get pipeline object 

In [ ]:
pipeline = client.get_pipeline(pipeline_id=pipeline_id)

In [ ]:
data_source = pipeline.get_source()

In [ ]:
# Generate some data and send it to the pipeline. Store it locally to compare
n_events = 100
for i in range(n_events):
    event = random_datagen()
    data_source.publish(event)

In [ ]:
## Explore the pipeline on the web-UI

In [ ]:
pipeline_url = f"https://app.glassflow.dev/pipelines/{pipeline.id}"
display(pipeline_url)

In [ ]:
# See the transformed data on the webhook url (if you used a webhook testing site)

In [ ]:
webhookurl = pipeline.sink_config['url']
if webhookurl:
    parts = webhookurl.rsplit('/', 1)
    webhook_viewer = parts[0] + "/#!/" + parts[1]
    print(webhook_viewer)
    